# Final Project Report
### *Solving the N-Body Problem Numerically Using a Lagrangian Approach*

**Author:** Xander Carroll    
**Course:** Physics 5300, Autumn 2025  
**Date:** December 6, 2025

**Project Repository:** [Link](https://github.com/Xander-Carroll/PHYSICS5300-NBody-Final)

---

## 0. Library Imports

In [1]:
# External library includes
import numpy as np

## 1. Physical Problem

### A. Project Proposal

I will build a numerical solver for the N-Body gravitational problem. I will implement a timestep integrator such as leapfrog, verify the behavior on simple systems (e.g., two-body orbits), and then run larger simulations to examine stability and emergent structure. If time allows, I would like to examine the problem for large values of N (~$10^4$ bodies). The resulting trajectories will be visualized using Manim or a similar animation tool. All code, animations, and documentation will be organized in a GitHub repository. There are a large number of resources (papers and pre-existing projects) for this problem. There is also great documentation for the python libraries we have been using.

### B. Describe the System

PLOTS SHOWING THE SYSTEM WITH COORDINATES ETC, ETC

In [ ]:
# Code to generate descriptive plots.

### C.1. Determining the Lagrangian

The potential and kinetic energy of the system needs to be described in terms of $\mathbf{r}_i$ and $\dot{\mathbf{r}}_i$ (the position and velocities of each body):

$$ T = \sum_{i=1}^{N} \dfrac{1}{2} m_i |\dot{\mathbf{r}_i}|^2 $$
$$ U = -G \sum_{i=1}^{N} \sum_{j>i}^{N} \dfrac{m_i m_j}{|\mathbf{r}_i - \mathbf{r}_j|}$$

Which we can then use to describe the Lagrangian:

$$ L = T - U = \sum_{i=1}^{N} \dfrac{1}{2} m_i |\dot{\mathbf{r}_i}|^2 + G \sum_{i=1}^{N} \sum_{j>i}^{N} \dfrac{m_i m_j}{|\mathbf{r}_i - \mathbf{r}_j|}$$

### C.2. Determining the Equations of Motion

We can find the following derivatives:

$$ \dfrac{\partial L}{\partial \mathbf{r}_i} = - G \sum_{j \ne i} \dfrac{m_i m_j (\mathbf{r}_i - \mathbf{r}_j)}{|\mathbf{r}_i - \mathbf{r}_j|^3}$$
$$ \dfrac{d}{dt} \dfrac{\partial L}{\partial \dot{\mathbf{r}}_i} = \dfrac{d}{dt} \big[m_i \dot{\mathbf{r}}_i\big] = m_i \ddot{\mathbf{r}}_i$$

Using the Euler-Lagrange equations for each coordinate body:

$$ \dfrac{d}{dt} \dfrac{\partial L}{\partial \dot{\mathbf{r}}_i} - \dfrac{\partial L}{\partial \mathbf{r}_i} = 0$$

Gives us the following Newtonian acceleration:

$$ m_i \ddot{\mathbf{r}}_i = - G \sum_{j \ne i} \dfrac{m_i m_j (\mathbf{r}_i - \mathbf{r}_j)}{|\mathbf{r}_i - \mathbf{r}_j|^3} $$

### D.1. Assumptions

The first assumption is an idealized system. The bodies will be treated as point masses, interacting solely via Newtonian gravitation. Relativistic effects, internal structure, collisions, and other forces will not be considered. 

The second assumption is dimensionality. We can simplify the problem by assuming that the orbits occur in a two-dimensional plane, rather than in three-dimensional space.


### D.2. Symmetries 

The Lagrangian has several symmetries:
- **Time:** because $L$ has no explicit dependence on $t$.
- **Translation:** because $L$ only depends on relative positions.
- **Rotation:** because rotating the system does not change $L$.

These symmetries imply that the *Energy*, *Linear Momentum*, and *Angular Momentum* of the system are conserved.



---

## 2. Visualizations

### A. Numerical Integration

The following is an implementation of leapfrog integration to solve the equations of motion numerically.